In [4]:
import numpy as np # NumPy is the fundamental package for scientific computing

import pandas as pd # Pandas is an easy-to-use data structures and data analysis tools
pd.set_option('display.max_columns', None) # To display all columns

import matplotlib.pyplot as plt # Matplotlib is a python 2D plotting library
%matplotlib inline 
# A magic command that tells matplotlib to render figures as static images in the Notebook.

import seaborn as sns # Seaborn is a visualization library based on matplotlib (attractive statistical graphics).
sns.set_style('whitegrid') # One of the five seaborn themes
import warnings
warnings.filterwarnings('ignore') # To ignore some of seaborn warning msg

from scipy import stats

from sklearn import linear_model # Scikit learn library that implements generalized linear models
from sklearn import neighbors # provides functionality for unsupervised and supervised neighbors-based learning methods
from sklearn.metrics import mean_squared_error # Mean squared error regression loss
from sklearn import preprocessing # provides functions and classes to change raw feature vectors

from math import log

In [35]:
WDS = [
    # 高德
    ('b33e77f31fe04969803d8c93eee90701','41204679123','导航下载', True,2.5),
    ('b33e77f31fe04969803d8c93eee90701','41204679138','语音导航', True,2.5),
    ('b33e77f31fe04969803d8c93eee90701','41204679141','手机导航', True,2.5),
    ('b33e77f31fe04969803d8c93eee90701','52066904374','导航地图下载', True,2.5),
    ('b33e77f31fe04969803d8c93eee90701','41204679126','手机导航下载', False,2.5),
    ('b33e77f31fe04969803d8c93eee90701','52257602639','手机导航地图',False,2.5),    
    ('b33e77f31fe04969803d8c93eee90701','41204679597','离线导航',False,2.5), 
    ('b33e77f31fe04969803d8c93eee90701','52257602606','手机导航下载',False,2.5),
    # 触宝
    ('99f34fe1ac3c472ca60f67296f2dabed','51851122225','有信电话免费下载', True,2.0),
    ('99f34fe1ac3c472ca60f67296f2dabed','51683066718','微会下载打电话免费', True,2.0),
    #('99f34fe1ac3c472ca60f67296f2dabed','52549511179','移动无限流量', True,2.0),
    ('99f34fe1ac3c472ca60f67296f2dabed','51683066700','微会', True,2.0),
    #('99f34fe1ac3c472ca60f67296f2dabed','52549510906','移动无限流量卡', True,2.0),
    ('99f34fe1ac3c472ca60f67296f2dabed','51662818937','网络电话', False,2.0),
    ('99f34fe1ac3c472ca60f67296f2dabed','51683068212','免费电话软件', False,2.0),
    ('99f34fe1ac3c472ca60f67296f2dabed','51683068299','免费电话', False,2.0),
    ('99f34fe1ac3c472ca60f67296f2dabed','51683067966','网络电话永久免费打', False,2.0),
    ('99f34fe1ac3c472ca60f67296f2dabed','51662819099','网络电话软件', False,2.0),
]

def simple_linear_model(train, test, input_feature):
    regr = linear_model.LinearRegression() # Create a linear regression object
    regr.fit(train.as_matrix(columns = [input_feature]), train.as_matrix(columns = ['profit'])) # Train the model
    RMSE = mean_squared_error(test.as_matrix(columns = ['profit']), 
                              regr.predict(test.as_matrix(columns = [input_feature])))**0.5 # Calculate the RMSE on test data
    return RMSE, regr.intercept_[0], regr.coef_[0][0]

# A function that take multiple features as input and return the RMSE (of the test data), and the  intercept and coefficients
def multiple_regression_model(train, test, input_features):
    regr = linear_model.LinearRegression() # Create a linear regression object
    regr.fit(train.as_matrix(columns = input_features), train.as_matrix(columns = ['profit'])) # Train the model
    RMSE = mean_squared_error(test.as_matrix(columns = ['profit']), 
                              regr.predict(test.as_matrix(columns = input_features)))**0.5 # Calculate the RMSE on test data
    return RMSE, regr.intercept_[0], regr.coef_ 

# A function that take multiple features as input and return the RMSE (of the train and validation data)
def RMSE(train, validation, features, new_input):
    features_list = list(features)
    features_list.append(new_input)
    regr = linear_model.LinearRegression() # Create a linear regression object
    regr.fit(train.as_matrix(columns = features_list), train.as_matrix(columns = ['price'])) # Train the model
    RMSE_train = mean_squared_error(train.as_matrix(columns = ['price']), 
                              regr.predict(train.as_matrix(columns = features_list)))**0.5 # Calculate the RMSE on train data
    RMSE_validation = mean_squared_error(validation.as_matrix(columns = ['price']), 
                              regr.predict(validation.as_matrix(columns = features_list)))**0.5 # Calculate the RMSE on train data
    return RMSE_train, RMSE_validation 

for appid,kid,kname,rtb,revenue in WDS:
    if not rtb:
        continue
    if kid not in ['41204679138',]:
        continue
    print(kname,kid)
    data = pd.read_csv("%s.csv" % kid) # load the data into a pandas dataframe


    data.drop([
                'time_created',
                'chgange_id',
                'keyword',
                'keywordid',
                #'quality',
                #'time_hour',
                #'price',
                #'rank1rate',
                #'baidu_impression',
                'btn_impression',
                'baidu_click',
                'btn_click',
                'baidu_cost',
                'btn_cost',
                'baidu_cpc',
                'btn_cpc',
                'baidu_impression_increase_rate',
                'btn_impression_increase_rate',
                'baidu_click_increase_rate',
                'btn_click_increase_rate',
                'baidu_cost_increase_rate',
                'btn_cost_increase_rate',
                'h5_dl',
                'dl_total',
                'cost_per_dl',
                'hour_baidu_click',
                'hour_btn_click',
                #'profit'
              ], 
              axis = 1, inplace = True)
    #data.drop(['changeid', 'time','hourclick'], axis = 1, inplace = True)
    #data.head(2) # Show the first 2 lines
    #print(data.head(2))
    
    if False:
        dummies_zipcodes = pd.get_dummies(data['keywordid'], drop_first=False)
        dummies_zipcodes.reset_index(inplace=True)
        dummies_zipcodes = dummies_zipcodes.add_prefix("{}#".format('keywordid'))
        dummies_zipcodes = dummies_zipcodes[['keywordid#41204679123',
                                         'keywordid#41204679138',
                                         'keywordid#41204679141',
                                         'keywordid#51851122225',
                                         'keywordid#51683066718',
                                         'keywordid#52549511179',
                                         'keywordid#51683066700',
                                         'keywordid#52549510906',
                                        ]]
        data.drop('keywordid', axis=1, inplace=True)
        data = data.join(dummies_zipcodes)

    data.dtypes

    from sklearn.cross_validation import train_test_split
    train_data, test_data = train_test_split(data, train_size = 0.8, random_state = 10)
    # A function that take one input of the dataset and return the RMSE (of the test data), and the intercept and coefficient

    input_list = data.columns.values.tolist() # list of column name
    input_list.remove('profit')
    simple_linear_result = pd.DataFrame(columns = ['feature', 'RMSE', 'intercept', 'coefficient'])

    # 查看所有参数的线性模型参数和RMSE
    if  False:
        # loop that calculate the RMSE of the test data for each input 
        for p in input_list:
            RMSE1, w1, w0 = simple_linear_model(train_data, test_data, p)
            simple_linear_result = simple_linear_result.append({'feature':p, 'RMSE':RMSE1, 'intercept':w0, 'coefficient': w1}
                                                       ,ignore_index=True)
        
        aa = simple_linear_result.sort_values('RMSE').head(15) # display the 10 best estimators
        print(aa)

    if True:
        train_data['time_hour_2'] = train_data['time_hour'].apply(lambda x: x**2)
        test_data['time_hour_2'] = test_data['time_hour'].apply(lambda x: x**2)

        train_data['price_2'] = train_data['price'].apply(lambda x: x**2)
        test_data['price_2'] = test_data['price'].apply(lambda x: x**2) 

        train_data['rank1_2'] = train_data['rank1rate'].apply(lambda x: x**2)
        test_data['rank1_2'] = test_data['rank1rate'].apply(lambda x: x**2)
    
        train_data['quality_2'] = train_data['quality'].apply(lambda x: x**2)
        test_data['quality_2'] = test_data['quality'].apply(lambda x: x**2)
    
        train_data['baidu_impression_2'] = train_data['baidu_impression'].apply(lambda x: x**2)
        test_data['baidu_impression_2'] = test_data['baidu_impression'].apply(lambda x: x**2)

        print('kid-%s' % kid)

    
        
        train_data_2, validation_data = train_test_split(train_data, train_size = 0.75, random_state = 50)
        input_list = train_data_2.columns.values.tolist() # list of column name
        input_list.remove('profit')

        # list of features included in the regression model and the calculated train and validation errors (RMSE)
        regression_greedy_algorithm = pd.DataFrame(columns = ['feature', 'train_error', 'validation_error'])  
        i = 0
        temp_list = []

        # a while loop going through all the features in the dataframe
        while i < len(train_data_2.columns)-1: 
            # a temporary dataframe to select the best feature at each iteration
            temp = pd.DataFrame(columns = ['feature', 'train_error', 'validation_error'])
    
            # a for loop to test all the remaining features
            for p in input_list:
                RMSE_train, RMSE_validation = RMSE(train_data_2, validation_data, temp_list, p)
                temp = temp.append({'feature':p, 'train_error':RMSE_train, 'validation_error':RMSE_validation}, ignore_index=True)
        
            temp = temp.sort_values('train_error') # select the best feature using train error
            best = temp.iloc[0,0]
            temp_list.append(best)
            regression_greedy_algorithm = regression_greedy_algorithm.append({'feature': best, 
                                                  'train_error': temp.iloc[0,1], 'validation_error': temp.iloc[0,2]}, 
                                                 ignore_index=True) # add the feature to the dataframe
            input_list.remove(best) # remove the best feature from the list of available features
            i += 1
        print(regression_greedy_algorithm)
    
        print('\n')
        greedy_algo_features_list = regression_greedy_algorithm['feature'].tolist()[:8] # select the first 30 features
        test_error, _, _ = multiple_regression_model(train_data_2, test_data, greedy_algo_features_list)
        print ('test error (RMSE) is: %s' %test_error)

        print('------------------\n')
        #regr = linear_model.LinearRegression() # Create a linear regression object
        #regr.fit(train.as_matrix(columns = greedy_algo_features_list), train.as_matrix(columns = ['profit']))
        aaa=train_data_2.as_matrix(columns = greedy_algo_features_list)
        print(aaa)
        #regr.predict(train.as_matrix(columns = features_list))
  #  break

语音导航 41204679138
kid-41204679138
              feature   train_error  validation_error
0               price  5.233642e-17      0.000000e+00
1           time_hour  3.700743e-17      0.000000e+00
2  baidu_impression_2  0.000000e+00      2.775558e-17
3             quality  5.233642e-17      0.000000e+00
4    baidu_impression  5.233642e-17      5.551115e-17
5           quality_2  5.233642e-17      5.551115e-17
6         time_hour_2  7.282570e-16      1.093443e-15
7             price_2  5.780746e-16      1.101865e-15
8           rank1rate  1.127361e-15      3.051599e-15
9             rank1_2  7.609591e-13      6.422024e-12


test error (RMSE) is: 15.7145266946
------------------

[[  6.70000000e-01   1.80000000e+01   1.60000000e+03   1.00000000e+01
    4.00000000e+01   1.00000000e+02   3.24000000e+02   4.48900000e-01]
 [  7.70000000e-01   1.80000000e+01   8.64900000e+03   1.00000000e+01
    9.30000000e+01   1.00000000e+02   3.24000000e+02   5.92900000e-01]
 [  6.00000000e-01   2.30000000e+